<a href="https://colab.research.google.com/github/yellayujin/studying/blob/main/240131_bs4_request.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import bs4

print(requests.__version__)
print(bs4.__version__)

2.31.0
4.12.3


## 네이버 상태 코드 확인

In [ ]:
URL = 'http://www.naver.com/'

req = requests.get(URL) # 검사 -> Network -> 목록 중 선택 -> Headers 등 선택하면 request method가 GET으로 나와있음 (POST방식이면 코드 조금 다르니 공식문서 찾아보길)
print(req.status_code)

200


In [ ]:
# req.text

In [ ]:
from bs4 import BeautifulSoup

# step 01: 데이터 수집
with open('index.html', 'r', encoding='UTF-8') as f:
    contents = f.read()

# step 02: 데이터 파싱(=순수한 html파일(문자열 형태)을 BeautifulSoup 객체로 변환 for 태그에 접근)
    soup = BeautifulSoup(contents, 'lxml')

    # print(soup)
    print(soup.h2)      # <h2>태그부분만 나옴
    print('--------')
    print(soup.ul)
    print('--------')
    print(soup.ul.li)

<h2>모바일 컴퍼니</h2>
--------
<ul id="mylist" style="width:150px">
<li>애플</li>
<li>삼성</li>
<li>노키아</li>
<li>LG</li>
</ul>
--------
<li>애플</li>


In [ ]:
print(soup.li)  # 위 문서에서 li코드 4개인데 하나만 나옴

<li>애플</li>


In [ ]:
# step 03: 데이터 수집을 위한 특정 태그 찾기
# 4개의 li태그에 있는 회사명을 모두 가져오는 것이 목적
companies = []
for tag in soup.find_all('li'):   # step 04: 데이터 가공
    companies.append(tag.text)   # .text로 태그는 안 가져오고 텍스트만 가져옴

print(companies)

['애플', '삼성', '노키아', 'LG']


In [ ]:
# step 05: 처리된 데이터 저장 pandas 데이터프레임
import pandas as pd
crawling_dict = {'회사명': companies}
result = pd.DataFrame(crawling_dict)
print(result)

   회사명
0   애플
1   삼성
2  노키아
3   LG


In [ ]:
# step 06: csv (or DB)로 내보내기)
result.to_csv('result.csv', index=False)

In [ ]:
# 벅스
custom_header = {
	'referer' : 'https://music.bugs.co.kr/',
	'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}
url = "https://music.bugs.co.kr/chart" # 크롤링 하려는 웹사이트
req = requests.get(url, headers = custom_header)

data = req.text
soup = BeautifulSoup(data, 'lxml')

In [ ]:
song_list = []
for tag in soup.find_all('p', class_='title'):
    song_list.append(tag.text.strip('\n'))

print(song_list)

['Love wins all', 'Wife', 'To. X', 'Love 119', '에피소드', 'Yes or No (Feat. 허윤진 of LE SSERAFIM, Crush)', 'MANIAC', 'Perfect Night', '비의 랩소디', 'Drama', 'Get A Guitar', '첫 만남은 계획대로 되지 않아', 'DASH', 'You & Me', 'I AM', '그대만 있다면 (여름날 우리 X 너드커넥션 (Nerd Connection))', 'Ditto', '한 페이지가 될 수 있게', 'Super Lady', 'Baddie', '헤어지자 말해요', '인사', 'ETA', '꿈', 'Hype Boy', '안 아름답고도 안 아프구나', '후라이의 꿈', 'Super Shy', 'Love Lee', 'Talk Saxy', 'Steal The Show (From “엘리멘탈”)', '그대가 내 안에 박혔다', 'Discord', 'Seven (feat. Latto) - Clean Ver.', 'Standing Next to You', '첫 눈', '숲', '너의 모든 순간', 'GODS', '예뻤어', 'Spicy', 'Attention', '넌 쉽게 말했지만', 'Chill Kill', 'Off The Record', '모든 날, 모든 순간 (Every day, Every Moment)', 'Kitsch', '별 떨어진다 (I Do)', '잠시라도 우리', 'New Jeans', '어떻게 이별까지 사랑하겠어, 널 사랑하는 거지', '퀸카 (Queencard)', 'Off My Face', 'Dangerously', '사건의 지평선', 'OMG', 'I Don’t Think That I Like Her', '사랑인가 봐', '새벽 통화', 'LOVE DIVE', '이브, 프시케 그리고 푸른 수염의 아내', '잘 지내자, 우리 (여름날 우리 X 로이킴)', '사막에서 꽃을 피우듯', 'UNFORGIVEN (feat. Nile Rodgers)', '어른'

In [ ]:
song_dict = {'곡 제목':  song_list}
result = pd.DataFrame(song_dict)
print(result)

                           곡 제목
0                 Love wins all
1                          Wife
2                         To. X
3                      Love 119
4                          에피소드
..                          ...
95  머물러주오 (Prod. 안신애 & Philtre)
96                       운이 좋았지
97                      Snowman
98                Cool With You
99         사랑은 먼 길을 돌아온 메아리 같아서

[100 rows x 1 columns]


In [ ]:
# 강사
import requests
from bs4 import BeautifulSoup
import pandas as pd

def crawling(soup) :
    # print(soup)
    tbody = soup.find("tbody")   # 지금 사이트에서는 table이 한 개지만, 여러 개일 경우 tbody부터 찾아야 해서 이렇게 작성
    result = []
    for p in tbody.find_all('p', class_ = 'title'):   # class라고만 쓰면 클래스 정의할 때 그 클래스랑 겹쳐서 class_로 정의함
        result.append(p.get_text().strip())
    return result

def main() :
    custom_header = {
        'referer' : 'https://music.bugs.co.kr/',
        'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    url = "https://music.bugs.co.kr/chart" # 크롤링 하려는 웹사이트
    req = requests.get(url, headers = custom_header)

    soup = BeautifulSoup(req.text, "html.parser")
    crawling(soup)

    titles = crawling(soup)
    print(pd.DataFrame({"노래제목" : titles}))


if __name__ == "__main__" :
    main()

                              노래제목
0                    Love wins all
1                             Wife
2                         Love 119
3                            To. X
4                             에피소드
..                             ...
95                         사랑을 하다가
96  Smoke (Prod. Dynamicduo, Padi)
97                        미워 (Ego)
98                           밤, 바다
99            사랑은 먼 길을 돌아온 메아리 같아서

[100 rows x 1 columns]


In [ ]:
# 주식 힌트
import requests
import pandas as pd
from bs4 import BeautifulSoup
import warnings
import time
import random

company_code = '005930' # 삼성전자
url ="https://finance.naver.com/item/sise_day.nhn?code=" + company_code   # 여기에 &page=숫자 들어감

headers = {
             'referer' : 'https://finance.naver.com/item/sise.naver?code=005930',
             'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
            }
req = requests.get(url, headers=headers)
soup = BeautifulSoup(req.content, "html.parser")

In [ ]:
pd.read_html(req.text, encoding='euc-kr')

C:\Users\skrtk\AppData\Local\Temp\ipykernel_9772\4192256325.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(req.text, encoding='euc-kr')


[            날짜       종가     전일비       시가       고가       저가         거래량
 0          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 1   2024.01.31  72800.0  1500.0  73400.0  74000.0  72600.0  12046907.0
 2   2024.01.30  74300.0   100.0  75000.0  75300.0  73700.0  12244418.0
 3   2024.01.29  74400.0  1000.0  73800.0  75200.0  73500.0  13976521.0
 4   2024.01.26  73400.0   700.0  73700.0  74500.0  73300.0  11160062.0
 5   2024.01.25  74100.0   100.0  74200.0  74800.0  73700.0  11737747.0
 6          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 7          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 8          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 9   2024.01.24  74000.0  1200.0  75200.0  75200.0  73500.0  12860661.0
 10  2024.01.23  75200.0   100.0  75700.0  75800.0  74300.0  14786224.0
 11  2024.01.22  75100.0   400.0  75900.0  76000.0  75000.0  19673375.0
 12  2024.01.19  74700.0  3000.0  73500.0  74700.0  73000.0  233

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import warnings
import time
import random
warnings.filterwarnings('ignore')

def crawling(url, headers, soup):
    last_page = int(soup.select_one('td.pgRR').a['href'].split('=')[-1])

    df = None
    count = 0
    for page in range(1, last_page + 1):
      req = requests.get(f'{url}&page={page}', headers=headers)
      df = pd.concat([df, pd.read_html(req.text, encoding = "euc-kr")[0]], ignore_index=True)
      if count > 10:
        break
      count += 1
      time.sleep( random.uniform(2,4))

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

def main():
    company_code = '005930' # 삼성전자
    url ="https://finance.naver.com/item/sise_day.nhn?code=" + company_code

    headers = {
             'referer' : 'https://finance.naver.com/item/sise.naver?code=005930',
             'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
            }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    result = crawling(url, headers, soup)
    print(result)

if __name__ == "__main__":
    main()

             날짜       종가     전일비       시가       고가       저가         거래량
0    2024.01.31  72700.0  1600.0  73400.0  74000.0  72500.0  13080752.0
1    2024.01.30  74300.0   100.0  75000.0  75300.0  73700.0  12244418.0
2    2024.01.29  74400.0  1000.0  73800.0  75200.0  73500.0  13976521.0
3    2024.01.26  73400.0   700.0  73700.0  74500.0  73300.0  11160062.0
4    2024.01.25  74100.0   100.0  74200.0  74800.0  73700.0  11737747.0
..          ...      ...     ...      ...      ...      ...         ...
115  2023.08.10  68000.0   900.0  68300.0  68500.0  67800.0  10227311.0
116  2023.08.09  68900.0  1300.0  68000.0  69600.0  67900.0  17259673.0
117  2023.08.08  67600.0   900.0  69000.0  69100.0  67400.0  14664709.0
118  2023.08.07  68500.0   200.0  67700.0  69200.0  67600.0  10968505.0
119  2023.08.04  68300.0   500.0  68800.0  69100.0  68200.0  12360193.0

[120 rows x 7 columns]
